## 默认绑定

使用 bind_providers 机制实现动运行时的变量传递。

每个 Runnable 实例都有一个 provider_dict 和 一个 consumer_dict，用于导出和导入绑定的变量。

如果 a 和 b 都是 Runnable 实例，并且执行了绑定操作 `a.bind_providers(b)`，就意味着将 a.consumer_dict 与 b.provider_dict 建立动态绑定关系。

这会导致：

1. 在 a 内部使用 a.consumer_dict 计算时会动态提取 b.provider_dict 的变量值
2. 默认情况下，会自动将 b.provider_dict 也同时作为 a.exported 的一部份传递，除非被同名导出变量覆盖

默认情况下，Runnable 实例不导入任何变量，但会导出 last_input 和 last_output 。

如果你需要增加导出变量，可以直接在 _provider_dict 字典中增加。

### 绑定实例

In [1]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        return f'你说了 {prompt}'

a = A()
a("hi")
print("a.provider_dict", a.provider_dict)

a.provider_dict {'last_output': '你说了 hi'}


### 绑定扩散行为
**这是一个默认行为：实例的 consumer_dict 会自动转为 provider_dict 向外扩散**

In [2]:
b = A()
b.bind_provider(a)
print("b's binding:", b.providers)
print("b.consumer_dict", b.consumer_dict)
print("b.provider_dict", b.provider_dict)

b's binding: [(<A.4392194240>, {})]
b.consumer_dict {'last_output': '你说了 hi'}
b.provider_dict {'last_output': '你说了 hi'}


### 使用实例本地值
**如果 consumer 修改了自己的 provider_dict 则用新值替代 provider 的输出值**

In [3]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        return f'你说了 {prompt}'
a = A()
a("hi")
b("我是 b")
print("b.consumer_dict", b.consumer_dict)
print("b.provider_dict", b.provider_dict)

b.consumer_dict {'last_output': '你说了 hi'}
b.provider_dict {'last_output': '你说了 我是 b'}


### 不传递 None 值
**实际上，上游 provider 中设置为 None 的值不会被传递**

In [4]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        return None

a = A(name="A")
a.call("hi")
print("a.consumer_dict", a.consumer_dict)
print("a.provider_dict", a.provider_dict)

a.consumer_dict {}
a.provider_dict {}


## 使用映射规则

**提供了映射规则之后，将按照新的键名传递变量。**

### 一般绑定

**默认情况下，映射后会修改上游 provider 键名称：**

In [5]:
from illufly.types import Runnable

class A(Runnable):
    def __init__(self, **kwargs):
        super().__init__()
        self._last_input = None

    def call(self, prompt: str):
        self._last_input = prompt
        return "OK"

    @property
    def provider_dict(self):
        local_dict = {"last_input": self._last_input}
        return {
            **super().provider_dict,
            **{k: v for k, v in local_dict.items() if v is not None}
        }

a = A()
c = A(name="C")
c.bind_provider(a, {"task": "last_input"})

a("hi")
print("a.provider_dict", a.provider_dict)
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

a.provider_dict {'last_output': 'OK', 'last_input': 'hi'}
c.consumer_dict {'last_output': 'OK', 'task': 'hi'}
c.provider_dict {'last_output': 'OK', 'task': 'hi'}


**也可以使用函数来扩展映射时的逻辑**

使用函数扩展时，不会覆盖函数中包含的键值，这实际上提供了 **1:N** 映射的可能性。

In [6]:
c = A()
c.bind_provider(a, {"task": lambda x: "我来自 @A " + x["last_input"]})
print("a.provider_dict", a.provider_dict)
print("c.consumer_dict", c.consumer_dict)

a.provider_dict {'last_output': 'OK', 'last_input': 'hi'}
c.consumer_dict {'last_output': 'OK', 'last_input': 'hi', 'task': '我来自 @A hi'}


In [7]:
c.providers

[(<A.4392194144>, {'task': <function __main__.<lambda>(x)>})]

### 禁止部份绑定键
**有时候不希望上游 provider_dict 某些键值覆盖本地，可以在映射时将其设置为 None：**

In [9]:
from illufly.types import Runnable

class A(Runnable):
    def __init__(self, **kwargs):
        super().__init__()
        self._last_input = None

    def call(self, prompt: str):
        self._last_input = prompt
        return "OK"

    @property
    def provider_dict(self):
        local_dict = {"last_input": self._last_input}
        return {
            **super().provider_dict,
            **{k: v for k, v in local_dict.items() if v is not None}
        }

a = A()
c = A(name="C")
c.bind_provider(a, {"last_input": None})

a("I am A")
c("I am C")
print("a.provider_dict", a.provider_dict)
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

a.provider_dict {'last_output': 'OK', 'last_input': 'I am A'}
c.consumer_dict {'last_output': 'OK'}
c.provider_dict {'last_output': 'OK', 'last_input': 'I am C'}


In [10]:
c.providers

[(<A.4656632000>, {'last_input': None})]

### 避免重复绑定

In [11]:
c.bind_provider(a, {"task": "last_input"})
print(a.consumers)
print(c.consumers)
print(a.providers)
print(c.providers)

[(<A.4656631952>, {'last_input': None})]
[]
[]
[(<A.4656632000>, {'last_input': None})]


In [12]:
a.bind_consumer(c, binding_map={"task": "last_input"})
print(a.consumers)
print(c.consumers)
print(a.providers)
print(c.providers)

[(<A.4656631952>, {'last_input': None})]
[]
[]
[(<A.4656632000>, {'last_input': None})]


### 绑定树

In [13]:
a.consumer_tree

{'provider': <A.4656632000>,
 'consumer_tree': [{'consumer': <A.4656631952>,
   'binding_map': {'last_input': None},
   'consumer_tree': {'provider': <A.4656631952>, 'consumer_tree': []}}]}

In [14]:
a.provider_tree

{'consumer': <A.4656632000>, 'provider_tree': []}

In [15]:
c.provider_tree

{'consumer': <A.4656631952>,
 'provider_tree': [{'provider': <A.4656632000>,
   'binding_map': {'last_input': None},
   'provider_tree': {'consumer': <A.4656632000>, 'provider_tree': []}}]}

### 动态绑定
有时候，仅希望绑定关系短暂维持。例如，在调用函数中临时建立的绑定关系，希望每次重置。<br>
**这与实例声明时希望长期建立的绑定关系不同，称为动态绑定。**

**首先，重新声明一个常规绑定：**

In [1]:
from illufly.types import Runnable

class A(Runnable):
    def call(self, prompt: str):
        return prompt

a = A(name="A")
c = A(name="C")
a("hi")

a.bind_consumer(c)
print("c.consumer_dict", c.consumer_dict)

c.consumer_dict {'last_output': 'hi'}


**紧接着，我们申请一个 c 的动态绑定：**

In [2]:
c.bind_provider(binding_map={"x": "I m x"}, dynamic=True)
c.consumer_dict

{'last_output': 'hi', 'x': 'I m x'}

**我们申请 一个 c 的动态绑定：**<br>
此时，上一次执行的动态绑定并不会造成干扰。

In [3]:
c.bind_provider({"y": "I m y"}, dynamic=True)
c.consumer_dict

{'last_output': 'hi', 'y': 'I m y'}

**如果绑定的是一个字典，会直接绑定字典的值**

In [4]:
c = A()
c.bind_provider({"task": "帮我写一首儿歌"})
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

c.consumer_dict {'task': '帮我写一首儿歌'}
c.provider_dict {'task': '帮我写一首儿歌'}


**如果绑定多个 Runnable 绑定相同变量，则以最后一个为准**

In [9]:
c = A()
c.bind_provider({"last_output": "帮我写一首儿歌"})
c.bind_provider(a)

a("啥是儿歌？")
print("c.consumer_dict", c.consumer_dict)
print("c.provider_dict", c.provider_dict)

c.consumer_dict {'last_output': '啥是儿歌？'}
c.provider_dict {'last_output': '啥是儿歌？'}


## 在容器实例内绑定

In [18]:
from illufly.types import Runnable

# A 有自己的计算函数，但绑定 B 之后可以将 B 的 provider_dict 作为入参实现动态 A 的动态计算
class A(Runnable):
    def call(self, prompt: str):
        yield "A: 今晚吃啥？\n"
        yield f'B: {self.consumer_dict["last_output"]}。'

class B(Runnable):
    def __init__(self, obj: Runnable):
        super().__init__()
        # 将 A 绑定到自己
        obj.bind_provider(self)
        self.obj = obj

    def call(self, instruction):
        return "猪肉炖粉条"

a = A()
b = B(a)
b("今晚吃啥？")
a("OH?")

A: 今晚吃啥？
B: 猪肉炖粉条。

In [20]:
print("a: ", a.consumer_dict, a.provider_dict)
print("b: ", b.consumer_dict, b.provider_dict)

a:  {'last_output': '猪肉炖粉条'} {'last_output': '猪肉炖粉条'}
b:  {} {'last_output': '猪肉炖粉条'}


## 使用 lazy_binding_map

使用 lazy_binding_map 可以先声明对象，然后在合适的时机自动实现绑定。

In [3]:
from illufly.types import PromptTemplate, Messages, Runnable

class A(Runnable):
    def call(self, prompt: str, **kwargs):
        return prompt

t = PromptTemplate(text="你的任务是：{{task}}", lazy_binding_map={"task": "last_output"})

a = A()
a.bind_consumer(t)

a("帮我写一首儿歌")
t.format()

'你的任务是：帮我写一首儿歌'